In [1]:
from fastai.vision.all import *
from timm import create_model
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
import torch
from sklearn.metrics import accuracy_score
from fastai.callback.mixup import *
from imblearn.over_sampling import RandomOverSampler

import torch.nn.functional as F
import torch
from torchmetrics.classification import Accuracy
from imblearn.under_sampling import RandomUnderSampler


SEED = 2023
N_FOLDS = 5
BATCH_SIZE = 20
IMGSZ = 384
EPOCHS = 10
INIT_LR = 2e-4
NUM_WORKER = 8
PATIENCE = 3
MODEL_BASE = 'convnext_large_in22k' # vit_small_patch16_224, convnext_base_fb_in22k


Train = 'Train (27).csv'
Test = 'Test (30).csv'

set_seed(SEED, reproducible=True)

2024-01-01 10:48:32.073189: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-01 10:48:32.074472: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-01 10:48:32.092870: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-01 10:48:32.092888: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-01 10:48:32.093438: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer


In [3]:

def prepare_train_data(data, kfold, image_dir):
    """
    Helper function to get the data ready
    """
    df = data.copy()
    df['image_id'] = df['filename'].apply(lambda x: x.split('.')[0])
    df = df.drop_duplicates(subset='image_id', keep='first')

    df['target'] = df['damage']

    df['fold'] = -1
    for i, (train_idx, val_idx) in enumerate(kfold.split(df, df['target'])):
        df.loc[val_idx, 'fold'] = i

    print(df.groupby(['fold', 'target']).size())

    df['path'] = df['filename'].apply(lambda x: f'{image_dir}/{x}')
    df['fold'] = df['fold'].astype('int')

    return df

In [4]:
def focal_loss(predictions, targets, alpha=0.25, gamma=2):
    predictions = predictions.sigmoid()
    
    # Convert targets to one-hot encoding
    targets_one_hot = F.one_hot(targets, num_classes=predictions.size(1)).float()

    focal_weight = alpha * targets_one_hot * (1 - predictions) ** gamma + (1 - alpha) * (1 - targets_one_hot) * predictions ** gamma
    loss = F.binary_cross_entropy_with_logits(predictions, targets_one_hot, reduction='none')
    
    return (focal_weight * loss).mean()


def train_model(data):
    df = data.copy()

    for fold in range(N_FOLDS):
        df['is_valid'] = (df['fold'] == fold)
        print(f'Training fold: {fold}')

        undersampler = RandomUnderSampler(sampling_strategy='auto', random_state=SEED)
        _, sample_indices = undersampler.fit_resample(df[df['fold'] != fold][['path']], df[df['fold'] != fold]['target'])
        df.loc[df.index.isin(sample_indices), 'undersampled'] = True
        df['undersampled'] = df['undersampled'].fillna(False).astype(bool)

        oversampler = RandomOverSampler(sampling_strategy='auto', random_state=SEED)
        _, sample_indices = oversampler.fit_resample(df[df['fold'] == fold][['path']], df[df['fold'] == fold]['target'])
        df.loc[df.index.isin(sample_indices), 'oversampled'] = True
        df['oversampled'] = df['oversampled'].fillna(False).astype(bool)

        pca_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('pca', PCA(n_components=10))
        ])

        preprocessor = ColumnTransformer(
            transformers=[
                ('pca', pca_transformer, ['path'])
            ])
        damage = pd.read_csv(Train).damage.unique()
        dblock = DataBlock(blocks=(ImageBlock, CategoryBlock(vocab=damage)),
                   get_x=ColReader('path'),
                   get_y=ColReader('target'),
                   splitter=ColSplitter('is_valid'),
                   item_tfms=Resize(IMGSZ),
                   batch_tfms=[*aug_transforms(mult=2),
                               Rotate(max_deg=20),
                               Flip(),
                               Brightness(max_lighting=0.6),
                               Contrast(max_lighting=0.6),
                               Zoom(max_zoom=1.1),
                               Warp(),
                               Dihedral(),
                               RandomErasing(p=0.2),
                               CropPad(size=128),
                               RandomResizedCrop(128),
                           ])
        dls = dblock.dataloaders(df,
                                bs=BATCH_SIZE,
                                num_workers=NUM_WORKER,
                                oversample_col='oversampled',
                                undersample_col='undersampled',
                                preproc_fn=preprocessor)

        model = MODEL_BASE
        learn = vision_learner(dls, model, loss_func=focal_loss, metrics=accuracy).to_fp16()

        learn.fine_tune(10, base_lr=INIT_LR)

        learn = learn.to_fp32()
        learn.save(f'{MODEL_BASE}_fold{fold}', with_opt=False)

In [5]:
train = pd.read_csv(Train)
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)


In [6]:
train_data = prepare_train_data(train, skf, 'images')
#train_model(train_data)

fold  target
0     DR         903
      G         2325
      ND          54
      WD        1848
      other       84
1     DR         904
      G         2324
      ND          54
      WD        1848
      other       84
2     DR         903
      G         2324
      ND          55
      WD        1848
      other       84
3     DR         903
      G         2325
      ND          55
      WD        1847
      other       83
4     DR         903
      G         2325
      ND          54
      WD        1847
      other       84
dtype: int64


In [7]:
####################################
TTA = 5
df = train_data
os.makedirs('submission', exist_ok=True)

test_df = pd.read_csv(Test)
test_df['path'] = test_df['filename'].map(lambda x: f'images/{x}')

ensemble = []
for fold in range(N_FOLDS):
    df['is_valid'] = (df['fold'] == fold)
    print(f'Training fold: {fold}')

    undersampler = RandomUnderSampler(sampling_strategy='auto', random_state=SEED)
    _, sample_indices = undersampler.fit_resample(df[df['fold'] != fold][['path']], df[df['fold'] != fold]['target'])
    df.loc[df.index.isin(sample_indices), 'undersampled'] = True
    df['undersampled'] = df['undersampled'].fillna(False).astype(bool)

    oversampler = RandomOverSampler(sampling_strategy='auto', random_state=SEED)
    _, sample_indices = oversampler.fit_resample(df[df['fold'] == fold][['path']], df[df['fold'] == fold]['target'])
    df.loc[df.index.isin(sample_indices), 'oversampled'] = True
    df['oversampled'] = df['oversampled'].fillna(False).astype(bool)

    pca_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('pca', PCA(n_components=10))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('pca', pca_transformer, ['path'])
        ])
    damage = pd.read_csv(Train).damage.unique()
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock(vocab=damage)),
                get_x=ColReader('path'),
                get_y=ColReader('target'),
                splitter=ColSplitter('is_valid'),
                item_tfms=Resize(IMGSZ),
                batch_tfms=[*aug_transforms(mult=2),
                            Rotate(max_deg=20),
                            Flip(),
                            Brightness(max_lighting=0.6),
                            Contrast(max_lighting=0.6),
                            Zoom(max_zoom=1.1),
                            Warp(),
                            Dihedral(),
                            RandomErasing(p=0.2),
                            CropPad(size=128),
                            RandomResizedCrop(128),
                        ])
    dls = dblock.dataloaders(df,
                            bs=BATCH_SIZE,
                            num_workers=NUM_WORKER,
                            oversample_col='oversampled',
                            undersample_col='undersampled',
                            preproc_fn=preprocessor)

    model = MODEL_BASE
    learn = vision_learner(dls, model, loss_func=focal_loss, metrics=accuracy).to_fp16()

    learn = learn.load(f'{MODEL_BASE}_fold{fold}')
    test_df['target'] = [1]*len(test_df)

    test_dl = dls.test_dl(test_df)
    preds, _ = learn.tta(dl=test_dl, n=TTA, beta=0)
    ensemble.append(preds.numpy())

Training fold: 0


/tmp/ipykernel_3811965/797184235.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df.index.isin(sample_indices), 'undersampled'] = True
/tmp/ipykernel_3811965/797184235.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df.index.isin(sample_indices), 'oversampled'] = True
/home/unicconaiadmin/.local/lib/python3.10/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name convnext_large_in22k to current convnext_large.fb_in22k.
  model = create_fn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 36.00 MiB (GPU 0; 23.69 GiB total capacity; 721.24 MiB already allocated; 2.94 MiB free; 788.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
test_df = test_df.join(pd.DataFrame(np.mean(ensemble, axis=0), columns=dls.vocab))

sample_submission_df = pd.read_csv("Sample Cnn.csv")
sample_submission_df = sample_submission_df['ID']
sample_submission_df = pd.merge(sample_submission_df, test_df, on='ID')
sample_submission_df = sample_submission_df[['ID']+dls.vocab]
sample_submission_df.to_csv(f"submission/{MODEL_BASE}_tta_{TTA}.csv", index=False)